In [72]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
import sys

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [29]:
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data\test.csv
data\train.csv


In [57]:
train_data = pd.read_csv('data/train.csv')
train_data.head()
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [64]:
clean_train_data = pd.read_csv('data/train.csv')
clean_train_data=clean_train_data.drop(columns=["Cabin"])
clean_train_data =clean_train_data.dropna(axis=0)
clean_train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000
mean,448.589888,0.404494,2.240169,29.642093,0.514045,0.432584,34.567251
std,258.683191,0.491139,0.836854,14.492933,0.930692,0.854181,52.938648
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,222.750000,0.000000,1.000000,20.000000,0.000000,0.000000,8.050000
50%,445.000000,0.000000,2.000000,28.000000,0.000000,0.000000,15.645850
75%,677.250000,1.000000,3.000000,38.000000,1.000000,1.000000,33.000000
max,891.000000,1.000000,3.000000,80.000000,5.000000,6.000000,512.329200


In [31]:
test_data = pd.read_csv("data/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [49]:
test_data.describe(include='all')

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,418.000000,418.000000,418,418,332.000000,418.000000,418.000000,418,417.000000,91,418
unique,NaN,NaN,418,2,NaN,NaN,NaN,363,NaN,76,3
top,NaN,NaN,"Sage, Mr. John George",male,NaN,NaN,NaN,PC 17608,NaN,B57 B59 B63 B66,S
freq,NaN,NaN,1,266,NaN,NaN,NaN,5,NaN,3,270
mean,1100.500000,2.265550,NaN,NaN,30.272590,0.447368,0.392344,NaN,35.627188,NaN,NaN
std,120.810458,0.841838,NaN,NaN,14.181209,0.896760,0.981429,NaN,55.907576,NaN,NaN
min,892.000000,1.000000,NaN,NaN,0.170000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,996.250000,1.000000,NaN,NaN,21.000000,0.000000,0.000000,NaN,7.895800,NaN,NaN
50%,1100.500000,3.000000,NaN,NaN,27.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,1204.750000,3.000000,NaN,NaN,39.000000,1.000000,0.000000,NaN,31.500000,NaN,NaN


In [99]:
y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch", "Ticket", "Fare","Embarked"]
X = pd.get_dummies(train_data[features])

train_X, val_X, train_y,val_y = train_test_split(X,y, random_state=1)
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(train_X, train_y)
predictions = model.predict(val_X)

acc = accuracy_score(predictions,val_y)

print(acc)

0.695067264573991


In [100]:
n_estimators = [10, 50, 75, 100, 150, 200, 250, 1000]
max_features = ['auto','sqrt', 'log2']
m_d_space = np.arange(5, 100,5)

# define grid search
params = dict(n_estimators=n_estimators,max_features=max_features,max_depth=m_d_space)

In [101]:
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=params, n_jobs=-1, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)

In [102]:
grid_result.best_score_, grid_result.best_params_

(0.8372732408511705,
 {'max_depth': 70, 'max_features': 'auto', 'n_estimators': 50})

In [97]:
test_hyp(150, 45, 'log2',train_X,val_X,train_y, val_y)

0.797752808988764

In [94]:
def test_hyp(n_e, m_d,m_f, tX,vX,ty,vy):
    model = RandomForestClassifier(n_estimators=n_e, max_depth=m_d, max_features=m_f,random_state=1)
    model.fit(tX, ty)
    predictions = model.predict(vX)

    return accuracy_score(predictions,vy)

In [95]:
n_e_space = [10, 50, 100, 150, 200, 250]
m_d_space = np.arange(5, 100,5)
m_f_space= ['auto','sqrt', 'log2']

In [96]:
mx = 0
best = []
for n_e in n_e_space:
    for m_d in m_d_space:
        for m_f in m_f_space:
            curr = test_hyp(n_e,m_d,m_f,train_X,val_X,train_y, val_y)
            if curr > mx:
                mx = curr
                best = [n_e,m_d,m_f]

In [98]:
print(best, test_hyp(best[0], best[1], best[2],train_X,val_X,train_y, val_y))

[10, 25, 'auto'] 0.8146067415730337
